In [1]:
#MAC USERS: 
#Before running ipynb - command line:
#ray stop
#ray start --head --port=6379 --redis-password="cbgt2"
#ray start --address='10.162.14.77:6379' --redis-password='cbgt2'
%load_ext autoreload
%autoreload 2
%reload_ext autoreload 
#if it doesn't work, shut down the ray server and start it again

In [2]:
!autopep8 --in-place --aggressive --aggressive init_params.py
!autopep8 --in-place --aggressive --aggressive popconstruct_copy.py

In [3]:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct_copy as popconstruct

2021-06-30 15:01:42,055	INFO worker.py:726 -- Connecting to existing Ray cluster at address: 10.162.14.77:6379


In [4]:
pl = cbgt.Pipeline()

In [5]:
#DEFAULTS - init_params.py 

def codeblock_celldefaults(self):
    self.celldefaults = par.helper_celldefaults() 
    
def codeblock_popspecific(self):
    self.popspecific = par.helper_popspecific() 
    
def codeblock_receptordefaults(self):
    self.receptordefaults = par.helper_receptordefaults()
    
def codeblock_basestim(self):
    self.basestim = par.helper_basestim()
    
def codeblock_dpmndefaults(self):
    self.dpmndefaults = par.helper_dpmndefaults()
    
def codeblock_d1defaults(self):
    self.d1defaults = par.helper_d1defaults()
    
def codeblock_d2defaults(self):
    self.d2defaults = par.helper_d2defaults()
    
def codeblock_actionchannels(self):
    self.actionchannels = par.helper_actionchannels()
    
#MODIFIERS - init_params.py 

def codeblock_modifycelldefaults(self):
    self.celldefaults_modified = par.modifier_celldefaults(self.celldefaults, self.params)
    
def codeblock_modifypopspecific(self):
    self.popspecific_modified = par.modifier_popspecific(self.popspecific, self.pops)
    
def codeblock_modifyreceptordefaults(self):
    self.receptordefaults_modified = par.modifier_receptordefaults(self.receptordefaults, self.receps)

def codeblock_modifybasestim(self):
    self.basestim_modified = par.modifier_basestim(self.basestim, self.base)
    
def codeblock_modifydpmndefaults(self):
    self.dpmndefaults_modified = par.modifier_dpmndefaults(self.dpmndefaults, self.dpmns)
    
def codeblock_modifyd1defaults(self):
    self.d1defaults_modified = par.modifier_d1defaults(self.d1defaults, self.d1)
    
def codeblock_modifyd2defaults(self):
    self.d2defaults_modified = par.modifier_d2defaults(self.d2defaults, self.d2)

def codeblock_modifyactionchannels(self): 
    self.actionchannels_modified = par.modifier_actionchannels(self.actionchannels, self.channels)
    
#POPDATA + PATHWAYS - popconstruct_copy.py

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct()
    
def codeblock_poppathways(self): 
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways) 

def codeblock_connAMPA(self): 
    self.connectivity_AMPA, self.meaneff_AMPA = popconstruct.helper_connectivityAMPA(self.popdata, self.pathways)
    
def codeblock_connGABA(self): 
    self.connectivity_GABA, self.meaneff_GABA = popconstruct.helper_connectivityGABA(self.popdata, self.pathways) 

def codeblock_connNMDA(self): 
    self.connectivity_NMDA, self.meaneff_NMDA = popconstruct.helper_connectivityNMDA(self.popdata, self.pathways)    

In [6]:
pl.add(codeblock_celldefaults)
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_popspecific)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_receptordefaults)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_basestim)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_dpmndefaults)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_d1defaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_d2defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_actionchannels)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)
pl.add(codeblock_connAMPA)
pl.add(codeblock_connGABA)
pl.add(codeblock_connNMDA)

In [7]:
pl.modulelist

In [8]:
environment = {
    'params': {'N':1, 'C':40},
    'pops' : {'FSI': {'Taum':60}}, 
    'receps' : {'Tau_AMPA': 100, 'Tau_GABA' : 100}, 
    'base' : {'FSI': {'FreqExt_AMPA': 100}}, 
    'dpmns' : {'dpmn_wmax': 400, 'dpmn_tauDOP' : 100},
    'd1': {'dpmn_type' : 3, 'dpmn_c' : 50},
    'd2' : {'dpmn_type' : 3, 'dpmn_c' : 50}, 
    'channels' : {'action' : [3,3]},
    'newpathways' : pd.DataFrame([['LIP', 'D1STR', 'AMPA', 'syn', 700, 0.027]], columns=['src', 'dest', 'receptor', 'type', 'con', 'eff'])
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [9]:
results.keys()

dict_keys(['params', 'pops', 'receps', 'base', 'dpmns', 'd1', 'd2', 'channels', 'newpathways', 'celldefaults', 'celldefaults_modified', 'popspecific', 'popspecific_modified', 'receptordefaults', 'receptordefaults_modified', 'basestim', 'basestim_modified', 'dpmndefaults', 'dpmndefaults_modified', 'd1defaults', 'd1defaults_modified', 'd2defaults', 'd2defaults_modified', 'actionchannels', 'actionchannels_modified', 'popdata', 'pathways', 'connectivity_AMPA', 'meaneff_AMPA', 'connectivity_GABA', 'meaneff_GABA', 'connectivity_NMDA', 'meaneff_NMDA'])

In [22]:
results['basestim']

{'FSI': {'FreqExt_AMPA': 3.6, 'MeanExtEff_AMPA': 1.55, 'MeanExtCon_AMPA': 800},
 'LIPI': {'FreqExt_AMPA': 1.05,
  'MeanExtEff_AMPA': 1.2,
  'MeanExtCon_AMPA': 640},
 'GPi': {'FreqExt_AMPA': 0.8, 'MeanExtEff_AMPA': 5.9, 'MeanExtCon_AMPA': 800},
 'STNE': {'FreqExt_AMPA': 4.45,
  'MeanExtEff_AMPA': 1.65,
  'MeanExtCon_AMPA': 800},
 'GPeP': {'FreqExt_AMPA': 4,
  'MeanExtEff_AMPA': 2,
  'MeanExtCon_AMPA': 800,
  'FreqExt_GABA': 2,
  'MeanExtEff_GABA': 2,
  'MeanExtCon_GABA': 2000},
 'D1STR': {'FreqExt_AMPA': 1.3, 'MeanExtEff_AMPA': 4, 'MeanExtCon_AMPA': 800},
 'D2STR': {'FreqExt_AMPA': 1.3, 'MeanExtEff_AMPA': 4, 'MeanExtCon_AMPA': 800},
 'LIP': {'FreqExt_AMPA': 2.2, 'MeanExtEff_AMPA': 2, 'MeanExtCon_AMPA': 800},
 'Th': {'FreqExt_AMPA': 2.2, 'MeanExtEff_AMPA': 2.5, 'MeanExtCon_AMPA': 800}}

In [24]:
results['basestim_modified']

{'FSI': {'FreqExt_AMPA': 100, 'MeanExtEff_AMPA': 1.55, 'MeanExtCon_AMPA': 800},
 'LIPI': {'FreqExt_AMPA': 1.05,
  'MeanExtEff_AMPA': 1.2,
  'MeanExtCon_AMPA': 640},
 'GPi': {'FreqExt_AMPA': 0.8, 'MeanExtEff_AMPA': 5.9, 'MeanExtCon_AMPA': 800},
 'STNE': {'FreqExt_AMPA': 4.45,
  'MeanExtEff_AMPA': 1.65,
  'MeanExtCon_AMPA': 800},
 'GPeP': {'FreqExt_AMPA': 4,
  'MeanExtEff_AMPA': 2,
  'MeanExtCon_AMPA': 800,
  'FreqExt_GABA': 2,
  'MeanExtEff_GABA': 2,
  'MeanExtCon_GABA': 2000},
 'D1STR': {'FreqExt_AMPA': 1.3, 'MeanExtEff_AMPA': 4, 'MeanExtCon_AMPA': 800},
 'D2STR': {'FreqExt_AMPA': 1.3, 'MeanExtEff_AMPA': 4, 'MeanExtCon_AMPA': 800},
 'LIP': {'FreqExt_AMPA': 2.2, 'MeanExtEff_AMPA': 2, 'MeanExtCon_AMPA': 800},
 'Th': {'FreqExt_AMPA': 2.2, 'MeanExtEff_AMPA': 2.5, 'MeanExtCon_AMPA': 800}}